# Notatki do pracowni drugiej

Rozwiązujemy układ równań

\begin{equation}
(x-x_1)^2 + (y-y_1)^2 + (z-z_1)^2 = \big[c(t_1-t)\big]^2 \\
(x-x_2)^2 + (y-y_2)^2 + (z-z_2)^2 = \big[c(t_2-t)\big]^2 \\
(x-x_3)^2 + (y-y_3)^2 + (z-z_3)^2 = \big[c(t_3-t)\big]^2 \\
(x-x_4)^2 + (y-y_4)^2 + (z-z_4)^2 = \big[c(t_4-t)\big]^2 \\
\end{equation}

Niech
\begin{equation}
f_i(x, y, z, t) = (x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2 - \big[c(t_i-t)\big]^2
\end{equation}

Z uogólnionego twierdzenia Taylora mamy
\begin{equation}
f_i(x + h_x, y + h_y, z + h_z, t + h_t) \approx f_i(x, y, z, t) + h_x \frac{\partial f_i}{\partial x}(x, y, z, t) 
+ h_y \frac{\partial f_i}{\partial y}(x, y, z, t) + h_z \frac{\partial f_i}{\partial z}(x, y, z, t)
+ h_t \frac{\partial f_i}{\partial t}(x, y, z, t)
\end{equation}

Ponadto
\begin{equation}
\frac{\partial f_i}{\partial x} = 2(x-x_i) \\
\frac{\partial f_i}{\partial y} = 2(y-y_i) \\
\frac{\partial f_i}{\partial z} = 2(z-z_i) \\
\frac{\partial f_i}{\partial t} = -2c^2(t-t_i)
\end{equation}

Stąd
\begin{equation}
f_i(x + h_x, y + h_y, z + h_z, t + h_t) \approx f_i(x, y, z, t) + 2h_x(x-x_i) + 2h_y(y-y_i) + 2h_z(z-z_i) -2c^2h_t(t-t_i)
\end{equation}

## Metoda Newtona

Zastosujemy metodę Newtona. Chcemy znaleźć $x, y, z, t$, dla których $f_i(x, y, z, t) = 0$ dla $1 \leq i \leq 4$.
Niech $x_n \in \mathbb{R}^4$ będzie n-tym przybliżeniem metody Newtona. Szukamy $h \in \mathbb{R}^4$, takiego że
dla $x_{n+1} = x_n + h$ zajdzie $f_i(x_{n+1}) = 0$ dla $1 \leq i \leq 4$.

Niech
$$
x_n = 
    \begin{pmatrix}
    x \\
    y \\
    z \\
    t
    \end{pmatrix}
$$

Rozwiążemy układ równań
\begin{equation}
0 = f_1(x, y, z, t) + 2h_x(x-x_1) + 2h_y(y-y_1) + 2h_z(z-z_1) -2c^2h_t(t-t_1) \\
0 = f_2(x, y, z, t) + 2h_x(x-x_2) + 2h_y(y-y_2) + 2h_z(z-z_2) -2c^2h_t(t-t_2) \\
0 = f_3(x, y, z, t) + 2h_x(x-x_3) + 2h_y(y-y_3) + 2h_z(z-z_3) -2c^2h_t(t-t_3) \\
0 = f_4(x, y, z, t) + 2h_x(x-x_4) + 2h_y(y-y_4) + 2h_z(z-z_4) -2c^2h_t(t-t_4)
\end{equation}
Inaczej
\begin{equation}
\begin{pmatrix}
x-x_1  & y-y_1 & z-z_1 & -c^2(t-t_1) \\
x-x_2  & y-y_2 & z-z_2 & -c^2(t-t_2) \\
x-x_3  & y-y_3 & z-z_3 & -c^2(t-t_3) \\
x-x_4  & y-y_4 & z-z_4 & -c^2(t-t_4) \\
\end{pmatrix}
\begin{pmatrix}
h_x \\ h_y \\ h_z \\ h_t
\end{pmatrix}
=
-\frac{1}{2}
\begin{pmatrix}
f_1(x, y, z, t) \\
f_2(x, y, z, t) \\
f_3(x, y, z, t) \\
f_4(x, y, z, t)
\end{pmatrix}
\end{equation}

Wtedy 
$$
x_{n+1} = x_n + h = x_n + \begin{pmatrix} h_x \\ h_y \\ h_z \\ h_t \end{pmatrix}
$$

In [293]:
c = 299792458.0

function fsingle(x, sat)
    a = (x - sat) .^ 2
    a[4] *= -c^2
    sum(a)
end 

function fvect(x, sats)
    [fsingle(x, sats[i]) for i = 1:length(sats)]
end

function getcoeffs(x, sats)
    A = [x - sats[i] for i = 1:length(sats)]
    A = hcat(A...)'
    A[:, 4] *= -c^2
    A
end

function newton(sats; maxiter::Int64=15) 
    x = [0, 0, 6370, 0]
    for i = 1:maxiter
        A = getcoeffs(x, sats)
        B = fvect(x, sats)
        h = -0.5 * (A \ B)
        x = x + h
    end
    x
end;

In [294]:
sat = Array{Array{Float64}}(4)
sat[1] = [15600,7540,20140, 0.07074e-3]
sat[2] = [18760,2750,18610, 0.07220e-3]
sat[3] = [17610,14630,13480, 0.07690e-3]
sat[4] = [19170,610,18390, 0.07242e-3]

newton(sat)

4-element Array{Float64,1}:
  -41.7727    
  -16.7892    
 6370.06      
   -3.20157e-6